In [1]:
from ScrapeFB import *
import pandas as pd
import bs4, time, re, json
from datetime import datetime, timedelta

In [2]:
window = WindowNoLogin(url='https://www.facebook.com/FreeYOUTHth/')

In [3]:
for i in range(50):
    window.scrape(filename='freeyouthFB.json', scroll=2)

100%|██████████| 2/2 [00:11<00:00,  5.76s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#pagelet_timeline_main_column"}
  (Session info: chrome=88.0.4324.96)


In [16]:
len(window.scraped_posts)

823

In [16]:
window = WindowLogin(
    url='https://www.facebook.com/FreeYOUTHth/',
    browser='Firefox',
    email='n700_rx8@yahoo.co.jp',
    password='Nozomi8931')

In [ ]:
for i in range(50):
    try:
        window.scrape(scroll=10)
        time.sleep(3)
    except:
        pass

100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


# show data

In [15]:
df = pd.read_json('FB.json')
df

,date,article,url,comment,share,hashtags
0,2021-08-01 21:14:00,ไล่ยิงประชาชนจนตอนนี้ยังไม่หยุด! กระสุนทุกลูก ...,https://www.facebook.com/FreeYOUTHth/posts/560...,3.3 พัน,1.4 หมื่น,[#ม็อบ7สิงหา]
1,2021-08-01 22:11:00,พวกมึงจะเอาให้ตายเลยใช่มั้ย? \n\n\nขอบคุณภาพจา...,https://www.facebook.com/FreeYOUTHth/posts/560...,3.1 พัน,2.1 หมื่น,[#ไอ้ฆาตกร]
2,2021-08-01 20:00:00,ยิงได้ยิงดี 7 สิงหานี้ เอากระสุนมึงกลับไป!\nพร...,https://www.facebook.com/FreeYOUTHth/posts/560...,827,6.5 พัน,"[#แค้นมีไว้ชำระ, #ม็อบ7สิงหา, #เอาห่ากระสุนมึง..."
3,2021-07-30 19:32:00,ขณะที่ประเทศกำลังฉิบหาย ผู้คนกำลังล้มตาย โจรกำ...,https://www.facebook.com/FreeYOUTHth/posts/559...,385,772,"[#ม็อบ7สิงหา, #เยาวชนปลดแอก]"
4,2021-07-29 23:07:00,พ.ต.อ.กฤษณะ พัฒนเจริญ รองโฆษกสำนักงานตำรวจแห่ง...,https://www.facebook.com/FreeYOUTHth/posts/559...,3.5 พัน,4 พัน,[#เยาวชนปลดแอก]
...,...,...,...,...,...,...
185,2021-03-05 18:30:00,"Freedom House จัดกลุ่มให้ไทยเป็นประเทศที่ ""ไม่...",https://www.facebook.com/FreeYOUTHth/posts/468...,947,9.5 พัน,[#เยาวชนปลดแอก]
186,2021-03-05 17:01:00,แม่คะ พวกริมสระมันเอาอีกแล้วค่ะ!\n6 มีนาคมนี้!...,https://www.facebook.com/FreeYOUTHth/posts/468...,235,363,"[#REDEM, #ริมสระ, #เยาวชนปลดแอกร่วมประชาสัมพันธ์]"
187,2021-03-05 16:10:00,มวลชน REDEM โหวตนัดหมาย 6 มีนาคมนี้! (ผ่านการโ...,https://www.facebook.com/FreeYOUTHth/posts/468...,1.1 พัน,2.9 พัน,[#REDEM]
188,2021-03-04 16:50:00,การประท้วงด้วยการเผารูปกษัตริย์เคยเกิดขึ้นมาแล...,https://www.facebook.com/FreeYOUTHth/posts/467...,2.5 พัน,1 หมื่น,[#เยาวชนปลดแอก]


In [5]:
df.to_csv('freeyouth.csv', index=False)